With this nodebook we intend to ingest a json list of AI related papers and output a summary by industry with the corresponding contributions we find within those papers. 

In [1]:
!pip install chromadb
!pip install hdbscan


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.9 MB/s eta 0:00:

In [2]:
import sys
import kagglehub 
gen_ai_capstone_utils_path = kagglehub.dataset_download('helgaguerreiro/gen-ai-capstone-utils')
sys.path.append(gen_ai_capstone_utils_path)

import json 
import os  
from utils import lib 

print(os.listdir(gen_ai_capstone_utils_path))

['utils', 'industry-list.json', 'papers_sample.json']


In [3]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai  
import shutil
#
# Config 
#
config = {
    "data_dir": gen_ai_capstone_utils_path, # where we store the input data 
    "out_dir":"/kaggle/working/",# where we are outputing intermediate data and the final document 
    "papers_file":"papers_sample.json", # what paper list are we ingesting 
    "GOOGLE_API_KEY": UserSecretsClient().get_secret("GOOGLE_API_KEY"),
    "reset": True
}

genai.configure(api_key=config['GOOGLE_API_KEY'])
papers = [] 

# 
# load the papers by reading the data/papers_2025-03-23_2025-03-29.json file
# if there's an intermediate file  load that one 
filepath = os.path.join(config['data_dir'],config['papers_file'])

if config.get('reset', False):
    files_to_remove = [
        '/kaggle/working/final_report.html',
        '/kaggle/working/papers_enriched.json',
        '/kaggle/working/suspicious_classifications.json',
        '/kaggle/working/industry_sections.json'
    ]
    
    for file_path in files_to_remove:
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Deleted file {file_path}")
        else:
            print(f"File {file_path} does not exist, skipping.")

    folder_path = '/kaggle/working/chromadb_industry'
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted folder {folder_path}")
    else:
        print(f"Folder {folder_path} does not exist, skipping.")

File /kaggle/working/final_report.html does not exist, skipping.
File /kaggle/working/papers_enriched.json does not exist, skipping.
File /kaggle/working/suspicious_classifications.json does not exist, skipping.
File /kaggle/working/industry_sections.json does not exist, skipping.
Folder /kaggle/working/chromadb_industry does not exist, skipping.


Make sure we can restaure intermediate results if something goes wrong while testing changes 

In [4]:
import pandas as pd 

if  os.path.exists(os.path.join(config['out_dir'],'papers_enriched.json')):
    filepath =  os.path.join(config['out_dir'],'papers_enriched.json')

print("Loading papers from:", filepath)
with open(filepath, 'r') as f:
    for line in f:
        papers.append(json.loads(line))

papers = pd.DataFrame(papers)
print("Loaded papers:", len(papers))


Loading papers from: /kaggle/input/gen-ai-capstone-utils/papers_sample.json
Loaded papers: 1905


We want to ground the model output to a pre-defined set of industries. 

The industry_meta contains 
* name - the industry name
* keywords - keywords associated with the industry
* examples - examples of what should match the industry

We will use this data to compose the prompt when querying the model. 

In [5]:
# Load the industry meta data from the data/industry-list.json file
with open(os.path.join(config['data_dir'], 'industry-list.json'), 'r') as f:
    industry_meta = json.load(f)

print(industry_meta)

[{'industry': 'Healthcare', 'keywords': ['doctor', 'hospital', 'medical', 'imaging', 'diagnosis', 'clinical'], 'examples': ['Detecting tumors from MRI scans.']}, {'industry': 'Finance & Banking', 'keywords': ['finance', 'banking', 'fraud', 'credit', 'insurance', 'investment', 'stock'], 'examples': ['Predicting credit default risk using machine learning.']}, {'industry': 'Education', 'keywords': ['education', 'student', 'learning', 'tutoring', 'school', 'curriculum'], 'examples': ['AI tutor for personalized mathematics education.']}, {'industry': 'Energy & Utilities', 'keywords': ['energy', 'grid', 'power', 'electricity', 'utility'], 'examples': ['Forecasting power demand in smart grids.']}, {'industry': 'Retail & E-commerce', 'keywords': ['retail', 'shopping', 'e-commerce', 'consumer', 'purchase', 'sales'], 'examples': ['Optimizing product recommendations for online stores.']}, {'industry': 'Agriculture', 'keywords': ['agriculture', 'crop', 'farming', 'irrigation', 'soil'], 'examples':

To have the model assign an industry to a paper we should consider the following: 
*When using flash models, it is best to separate the task of deciding if it is an industry-relevant paper from the task of deciding which industry fits 
* Grounding the prompt helps in directing the model to specific names of industries but does not prevent the model from hallucinating 
* Flash models have poor abstraction capabilities and will often drift into inferring industry associations even at low temperatures; we should always check the output of the model to exclude hallucinations 

* We instruct the model to assign the category Other as a fallback if none of the given allowed categories match 
* When the flash model returns three or more categories, or it returns the category "Other" alongside other categories, we classify this as a "suspicious classification" and  use a pro model to re-run the prompt and replace the flash model's assessment

implementation is defined in utils/industry.py

In [6]:
from utils import industry 
# Run each papers through the model so we can assign an industry label 
industry.classify_industry(papers,industry_meta,config['out_dir'])
print(papers.head(10))

################################################## 
ℹ️ Saving intermediate results...


 ################################################## 
Paper 1/1905: 2503.17894 Generative AI for Validating Physics Laws 


Paper is not related to industry development, skipping...


 ################################################## 
Paper 2/1905: 2503.17896 Multi-Disease-Aware Training Strategy for Cardiac MR Image Segmentation 


Paper is not related to industry development, skipping...


 ################################################## 
Paper 3/1905: 2503.17897 Real-time Global Illumination for Dynamic 3D Gaussian Scenes 


Time taken to query model: 1.8614380359649658 seconds
Industry list:
 [{'industry': 'Entertainment & Media', 'relevanceScore': 85, 'summary': 'This paper focuses on real-time rendering of dynamic 3D Gaussian scenes with global illumination. It presents a pipeline and algorithms for achieving high-quality lighting effects at interactive frame rates. This technology is high

In [7]:

# What % of papers are industry related , select those with industry N/A
industry_related = papers[
    papers['industry_list'].apply(lambda x: len(x) > 0 and x[0]['industry'] != 'N/A')
]
print("Papers related to industry development:", len(industry_related))
print("Percentage of papers related to industry development:", len(industry_related)/len(papers)*100)

Papers related to industry development: 967
Percentage of papers related to industry development: 50.76115485564304


Some industries will contain many papers; every summary implies information compression. We can increase resolution without exploding the summary size if we can cluster similar papers and evaluate each cluster separately. 

To cluster papers together, first, we create semantic embedings. 

In [8]:
import google.genai as genai_embed
from utils import embedings

collection = lib.get_chroma_collection(name="paper_abstracts", base_path=config['out_dir'])
sample_embeding = lib.get_chroma_record(collection,'2504.01981__Manufacturing')
if sample_embeding is not None:
    print("\n\n#" * 20, "ChromaDB collection already exists. Skipping embedding.", sample_embeding )
else:
    print("\n\n","#" * 20, "Embedding papers.", sample_embeding)
    client = genai_embed.Client(api_key=config['GOOGLE_API_KEY']) 
    embedings.embed_papers(client,papers)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(




 #################### Embedding papers. None
✅ Collection 'paper_abstracts' contains 0 items.
🧠 Found 17 industries to embed.
\n🛠️ Embedding 138 papers for industry: Entertainment & Media
🚀 Embedding 10 papers for Entertainment & Media: ['2503.17897__Entertainment & Media', '2503.17907__Entertainment & Media', '2503.17934__Entertainment & Media', '2503.17973__Entertainment & Media', '2503.17975__Entertainment & Media', '2503.17987__Entertainment & Media', '2503.18032__Entertainment & Media', '2503.18083__Entertainment & Media', '2503.18114__Entertainment & Media', '2503.18117__Entertainment & Media']
🚀 Embedding 10 papers for Entertainment & Media: ['2503.18155__Entertainment & Media', '2503.18159__Entertainment & Media', '2503.18168__Entertainment & Media', '2503.18190__Entertainment & Media', '2503.18282__Entertainment & Media', '2503.18286__Entertainment & Media', '2503.18324__Entertainment & Media', '2503.18331__Entertainment & Media', '2503.18349__Entertainment & Media', '2503.1

The goal is to create 1 section per industry. 

Each section contains multiple clusters and one “General Outlook" summary representing the papers that did not fit any clusters. 

We also ask the model to assign a representative title for each cluster.

In [9]:
from utils import summary
summary.get_summaries(papers,config['out_dir'])

🧠 Found 19 industries to summarize. ['Finance & Banking', 'Education', 'Construction & Real Estate', 'Transportation & Logistics', 'Public Policy & Governance', 'Energy & Utilities', 'Retail & E-commerce', 'Manufacturing', 'Other', 'Environmental Science', 'Entertainment & Media', 'Healthcare', 'N/A', 'Cybersecurity', 'Legal', 'Social Sciences & Humanities', 'Aerospace & Defense', 'Geospatial & Remote Sensing', 'Agriculture']


Summarizing industry: Finance & Banking 
get_sections for industry: Finance & Banking


Clustering papers for industry: Finance & Banking
            industry          id
0  Finance & Banking  2503.17909
1  Finance & Banking  2503.17963
2  Finance & Banking  2503.18096
3  Finance & Banking  2503.18224
4  Finance & Banking  2503.18235
5  Finance & Banking  2503.18265
6  Finance & Banking  2503.18313
7  Finance & Banking  2503.18317
8  Finance & Banking  2503.18434
9  Finance & Banking  2503.18596
🧠 Found 49 papers for industry: Finance & Banking with 6 clusters
C

[{'industry': 'Finance & Banking',
  'sections': [{'title': 'General Outlook',
    'summary': 'Research highlights the application of advanced AI models, including Transformers, Graph Neural Networks, and Large Language Models, to address specific challenges within Finance and Banking. Common contributions involve tailoring these sophisticated techniques for tasks such as automated trading strategy development using novel loss functions, enhanced risk assessment via anomaly detection on complex user data, and improved analysis of financial audio data through specialized benchmarks. There is a shared focus on methodological innovation to boost performance and robustness, demonstrated by new algorithms for high-frequency trading, refined federated learning frameworks addressing data heterogeneity, and specialized network architectures for risk detection. Rigorous evaluation against established benchmarks underscores the potential of these AI advancements to deliver superior solutions for

Finally, we want to output an HTML file that allows us to preview the data in a more user-friendly way.

In [10]:
from utils import report 
report.generate_html_report(
    os.path.join(config['out_dir'], 'industry_sections.json'),
    os.path.join(config['out_dir'], 'final_report.html')
)

✅ Report generated: /kaggle/working/final_report.html
